In [ ]:
# Importação das bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Configuração do estilo dos gráficos
sns.set(style="whitegrid")

# ========================================
# 1. Geração de Dados Fictícios
# ========================================
np.random.seed(42)
n_samples = 1000

data = {
    'area_m2': np.random.uniform(40, 300, n_samples),
    'quartos': np.random.randint(1, 6, n_samples),
    'banheiros': np.random.randint(1, 4, n_samples),
    'distancia_centro_km': np.random.uniform(0.5, 20, n_samples),
    'preco': np.zeros(n_samples)
}

df = pd.DataFrame(data)

# Simulação realista do preço (com ruído)
df['preco'] = (
    df['area_m2'] * 8000 +
    df['quartos'] * 30000 +
    df['banheiros'] * 40000 -
    df['distancia_centro_km'] * 15000 +
    np.random.normal(0, 50000, n_samples)
)

# ========================================
# 2. ETL e Limpeza
# ========================================
print("Informações do dataset:")
print(df.info())
print("\nPrimeiras linhas:")
print(df.head())

# Verificar valores ausentes
print("\nValores ausentes:")
print(df.isnull().sum())

# Estatísticas descritivas
print("\nEstatísticas descritivas:")
print(df.describe())

# ========================================
# 3. Visualizações
# ========================================
plt.figure(figsize=(15, 10))

# Preço vs Área
plt.subplot(2, 2, 1)
sns.scatterplot(x='area_m2', y='preco', data=df, alpha=0.6)
plt.title('Preço vs Área (m²)')
plt.xlabel('Área (m²)')
plt.ylabel('Preço (R$)')

# Preço vs Quartos
plt.subplot(2, 2, 2)
sns.boxplot(x='quartos', y='preco', data=df)
plt.title('Preço por Número de Quartos')

# Correlação
plt.subplot(2, 2, 3)
corr = df.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação')

# Distribuição do preço
plt.subplot(2, 2, 4)
sns.histplot(df['preco'], kde=True, bins=30)
plt.title('Distribuição dos Preços')

plt.tight_layout()
plt.show()

# ========================================
# 4. Modelo: Regressão Linear
# ========================================
X = df[['area_m2', 'quartos', 'banheiros', 'distancia_centro_km']]
y = df['preco']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

y_pred = model_lr.predict(X_test)

# ========================================
# 5. Avaliação
# ========================================
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n" + "="*50)
print("REGRESSÃO LINEAR - RESULTADOS")
print("="*50)
print(f"R²: {r2:.4f}")
print(f"RMSE: R$ {rmse:,.2f}")
print(f"MAE: R$ {mae:,.2f}")

# Coeficientes
print("\nCoeficientes do modelo:")
for feature, coef in zip(X.columns, model_lr.coef_):
    print(f"  {feature}: R$ {coef:,.2f}")
print(f"  Intercepto: R$ {model_lr.intercept_:,.2f}")

# Gráfico de predição vs real
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r  r--', lw=2)
plt.xlabel('Valor Real')
plt.ylabel('Valor Previsto')
plt.title('Regressão Linear: Real vs Previsto')
plt.show()